In [1]:
import pandas as pd
from pandas.io.json import json_normalize
from pandas import DataFrame
import os
from geopy import geocoders
from geopy.geocoders import GoogleV3
from config import gkey

import geopandas as gpd
from shapely import wkt
from shapely.wkt import loads
import re

In [2]:
# Pull in geojson address data from openaddresses
fname = "resources/us_il_city_of_evanston-addresses-city.geojson"

# read file to dataframe
gdf = gpd.read_file(fname)

simpledec = re.compile(r"\d*\.\d+")
def mround(match):
    return "{:.5f}".format(float(match.group()))

gdf.geometry = gdf.geometry.apply(lambda x: loads(re.sub(simpledec, mround, x.wkt)))
gdf

,id,unit,number,street,city,district,region,postcode,hash,geometry
0,,,811,CHICAGO AVE,EVANSTON,,IL,60202,0f1ae9d6dc3e261d,POINT (-87.67892 42.03261)
1,,,2730,CENTRAL ST,EVANSTON,,IL,60201,04d8292c6fe41f75,POINT (-87.71218 42.06420)
2,,,2726,CENTRAL ST,EVANSTON,,IL,60201,0f6aa09515f2553f,POINT (-87.71201 42.06420)
3,,,2720,CENTRAL ST,EVANSTON,,IL,60201,a9c1519f7bce1807,POINT (-87.71154 42.06419)
4,,,2722,CENTRAL ST,EVANSTON,,IL,60201,48834904e3e1b801,POINT (-87.71182 42.06420)
...,...,...,...,...,...,...,...,...,...,...
17130,,,1122,PITNER AVE,EVANSTON,,IL,60202,a06c3531387e9893,POINT (-87.70577 42.03833)
17131,,,1124,PITNER AVE,EVANSTON,,IL,60202,30eaae035c012e54,POINT (-87.70577 42.03839)
17132,,,1514,DEWEY AVE,EVANSTON,,IL,60201,c5f8058ef65e857f,POINT (-87.69672 42.04531)
17133,,,1141,DEWEY AVE,EVANSTON,,IL,60202,ba9fd44f5993932b,POINT (-87.69571 42.03897)


In [3]:
# Verify/Make pandas dataframe
addressdf = pd.DataFrame(gdf)

# Change 'geometry' column datatype from geometry to object
addressdf['geometry'] = addressdf.geometry.apply(lambda x: wkt.dumps(x, rounding_precision=5))


In [4]:
# Check dataframe
addressdf

,id,unit,number,street,city,district,region,postcode,hash,geometry
0,,,811,CHICAGO AVE,EVANSTON,,IL,60202,0f1ae9d6dc3e261d,POINT (-87.67892 42.03261)
1,,,2730,CENTRAL ST,EVANSTON,,IL,60201,04d8292c6fe41f75,POINT (-87.71218 42.06420)
2,,,2726,CENTRAL ST,EVANSTON,,IL,60201,0f6aa09515f2553f,POINT (-87.71201 42.06420)
3,,,2720,CENTRAL ST,EVANSTON,,IL,60201,a9c1519f7bce1807,POINT (-87.71154 42.06419)
4,,,2722,CENTRAL ST,EVANSTON,,IL,60201,48834904e3e1b801,POINT (-87.71182 42.06420)
...,...,...,...,...,...,...,...,...,...,...
17130,,,1122,PITNER AVE,EVANSTON,,IL,60202,a06c3531387e9893,POINT (-87.70577 42.03833)
17131,,,1124,PITNER AVE,EVANSTON,,IL,60202,30eaae035c012e54,POINT (-87.70577 42.03839)
17132,,,1514,DEWEY AVE,EVANSTON,,IL,60201,c5f8058ef65e857f,POINT (-87.69672 42.04531)
17133,,,1141,DEWEY AVE,EVANSTON,,IL,60202,ba9fd44f5993932b,POINT (-87.69571 42.03897)


In [5]:
# Combine address columns to one column
# addressdf['address'] = addressdf['street'].str.cat(addressdf['city'].str.cat(addressdf['region'],sep=" ")

# Remove extra strings from the latitude and longitude in the geometry column
addressdf['geometry'] = addressdf['geometry'].str.replace('POINT ', '').str.replace('(', '').str.replace(')', '')
addressdf

,id,unit,number,street,city,district,region,postcode,hash,geometry
0,,,811,CHICAGO AVE,EVANSTON,,IL,60202,0f1ae9d6dc3e261d,-87.67892 42.03261
1,,,2730,CENTRAL ST,EVANSTON,,IL,60201,04d8292c6fe41f75,-87.71218 42.06420
2,,,2726,CENTRAL ST,EVANSTON,,IL,60201,0f6aa09515f2553f,-87.71201 42.06420
3,,,2720,CENTRAL ST,EVANSTON,,IL,60201,a9c1519f7bce1807,-87.71154 42.06419
4,,,2722,CENTRAL ST,EVANSTON,,IL,60201,48834904e3e1b801,-87.71182 42.06420
...,...,...,...,...,...,...,...,...,...,...
17130,,,1122,PITNER AVE,EVANSTON,,IL,60202,a06c3531387e9893,-87.70577 42.03833
17131,,,1124,PITNER AVE,EVANSTON,,IL,60202,30eaae035c012e54,-87.70577 42.03839
17132,,,1514,DEWEY AVE,EVANSTON,,IL,60201,c5f8058ef65e857f,-87.69672 42.04531
17133,,,1141,DEWEY AVE,EVANSTON,,IL,60202,ba9fd44f5993932b,-87.69571 42.03897


In [6]:
# Remove un-needed columns
addressdf.drop(['unit', 'district'],axis=1,inplace=True)
addressdf

,id,number,street,city,region,postcode,hash,geometry
0,,811,CHICAGO AVE,EVANSTON,IL,60202,0f1ae9d6dc3e261d,-87.67892 42.03261
1,,2730,CENTRAL ST,EVANSTON,IL,60201,04d8292c6fe41f75,-87.71218 42.06420
2,,2726,CENTRAL ST,EVANSTON,IL,60201,0f6aa09515f2553f,-87.71201 42.06420
3,,2720,CENTRAL ST,EVANSTON,IL,60201,a9c1519f7bce1807,-87.71154 42.06419
4,,2722,CENTRAL ST,EVANSTON,IL,60201,48834904e3e1b801,-87.71182 42.06420
...,...,...,...,...,...,...,...,...
17130,,1122,PITNER AVE,EVANSTON,IL,60202,a06c3531387e9893,-87.70577 42.03833
17131,,1124,PITNER AVE,EVANSTON,IL,60202,30eaae035c012e54,-87.70577 42.03839
17132,,1514,DEWEY AVE,EVANSTON,IL,60201,c5f8058ef65e857f,-87.69672 42.04531
17133,,1141,DEWEY AVE,EVANSTON,IL,60202,ba9fd44f5993932b,-87.69571 42.03897


In [ ]:

# fltraddress = addressdf[addressdf['addresses'].str.lower().str.contains('^6')]
# fltraddress

In [8]:
# adds_sml = pd.read_csv("static/data/sample/adds_sml.csv")
def format_coordinates(x):
    coordinates = x.split(' ')
    lat = str(coordinates[1])
    long = str(coordinates[0])
    formatted = lat + ', ' + long
    return formatted
addressdf['geometry'] = addressdf['geometry'].apply(format_coordinates)
addressdf


,id,number,street,city,region,postcode,hash,geometry
0,,811,CHICAGO AVE,EVANSTON,IL,60202,0f1ae9d6dc3e261d,"42.03261, -87.67892"
1,,2730,CENTRAL ST,EVANSTON,IL,60201,04d8292c6fe41f75,"42.06420, -87.71218"
2,,2726,CENTRAL ST,EVANSTON,IL,60201,0f6aa09515f2553f,"42.06420, -87.71201"
3,,2720,CENTRAL ST,EVANSTON,IL,60201,a9c1519f7bce1807,"42.06419, -87.71154"
4,,2722,CENTRAL ST,EVANSTON,IL,60201,48834904e3e1b801,"42.06420, -87.71182"
...,...,...,...,...,...,...,...,...
17130,,1122,PITNER AVE,EVANSTON,IL,60202,a06c3531387e9893,"42.03833, -87.70577"
17131,,1124,PITNER AVE,EVANSTON,IL,60202,30eaae035c012e54,"42.03839, -87.70577"
17132,,1514,DEWEY AVE,EVANSTON,IL,60201,c5f8058ef65e857f,"42.04531, -87.69672"
17133,,1141,DEWEY AVE,EVANSTON,IL,60202,ba9fd44f5993932b,"42.03897, -87.69571"


In [9]:
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in addressdf['geometry']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
addressdf['latitude'] = lat
addressdf['longitude'] = lon

In [10]:
adds_sml = addressdf.sample(n = 100) 
adds_sml

,id,number,street,city,region,postcode,hash,geometry,latitude,longitude
3588,,1015,FOREST AVE,EVANSTON,IL,60202,f13a35dfe5ee13be,"42.03624, -87.67468",42.03624,-87.67468
7510,,3131,THAYER ST,EVANSTON,IL,60201,cb471cbce95f5a0c,"42.06768, -87.72222",42.06768,-87.72222
10212,,822,DODGE AVE,EVANSTON,IL,60202,e1b98ce70a52de77,"42.03287, -87.69975",42.03287,-87.69975
5007,,1742,LELAND AVE,EVANSTON,IL,60201,a29ffdcf86356e87,"42.04983, -87.70742",42.04983,-87.70742
6256,,2319,PARK PL,EVANSTON,IL,60201,1c0cf147f77505f7,"42.06700, -87.70576",42.06700,-87.70576
...,...,...,...,...,...,...,...,...,...,...
7171,,1119,SHERMAN AVE,EVANSTON,IL,60202,50383233b5bfd608,"42.03833, -87.68189",42.03833,-87.68189
2067,,825,CHURCH ST,EVANSTON,IL,60201,3ab1521433725654,"42.04862, -87.68289",42.04862,-87.68289
5768,,1106,MONROE ST,EVANSTON,IL,60202,164e80d6f813f17d,"42.03065, -87.68745",42.03065,-87.68745
7230,,1854,SHERMAN AVE,EVANSTON,IL,60201,b026a1f7674ad323,"42.05151, -87.68203",42.05151,-87.68203


In [11]:
# adds_sml.to_csv (r'static/data/sample/adds_sml.csv', index = True, header=True)

In [ ]:
# REPLACE 'space' with 'comma' in the geometry column
# adds_sml['geometry'] = adds_sml['geometry'].str.replace(' ',', ')
# adds_sml

In [ ]:
# '''addresses = ["2620 N Drake Ave, Chicago, Illinois 60647",
#          "2622 N Drake Ave, Chicago, Illinois 60647",
#          "2626 N Drake Ave, Chicago, Illinois 60647",
#          "2630 N Drake Ave, Chicago, Illinois 60647",
#          "2648 N Drake Ave, Chicago, Illinois 60647"]'''

In [ ]:
# '''df = DataFrame (addresses, columns=['Address'])
# df'''

In [ ]:
# *** IF WE ONLY HAVE ADDRESSES AND NEED TO CONVERT TO LAT LONG ***

# API_KEY = os.getenv("API1234")
# g = GoogleV3(api_key=gkey)


# loc_coordinates = []
# loc_address = []
# for address in df.Address:
#     try:
#         inputAddress = address
#         location = g.geocode(inputAddress, timeout=15)
#         loc_coordinates.append((location.latitude, location.longitude))
#         loc_address.append(inputAddress)
#     except Exception as e:
#         print('Error, skipping address...', e)


# df_geocodes = pd.DataFrame({'coordinate':loc_coordinates,'address':loc_address})
# df_geocodes

In [12]:
# GRAB A SERIES OF STREET VIEW IMAGES FROM latlong PROVIDED
import json
# Import google_streetview for the api module
import google_streetview.api
import time
import glob
import streetview
import itertools 


In [13]:
# Loop through the cities_pd and run a lat/long search for each city
geo_lat = adds_sml['latitude'].to_list()
geo_lon = adds_sml['longitude'].to_list()
i = 0
for (lat, lon) in zip(geo_lat, geo_lon):
    
    panoids = streetview.panoids(lat=lat, lon=lon)
    
    streetview.api_download(panoids, 90, "static/data/raw", gkey)



Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image not found
Image no

In [ ]:
# create a params dict that will be updated with new city each iteration
params = {
    'size': '640x640', # max 640x640 pixels
    'heading': '90',
    'location': '',
    'pitch': '-0.76',
    'fov': 120,
    'key': gkey
}

# Loop through the cities_pd and run a lat/long search for each city
geometry_list = adds_sml['geometry'].to_list()
i = 0
for latlong in geometry_list[50:60]:
    
    # update address key value
    params['location'] = latlong
    
    # Create a results object
#     results = google_streetview.api.results(params)
    results = google_streetview.api.results([params])
    
    
     

    # Download images to directory 'downloads'
    results.download_links(f"static/data/raw/geo{i}",)
    old_file_name = f"static/data/raw/geo{i}/gsv_0.jpg"
    new_file_name = f"static/data/raw/geo{i}/gsv_{i}.jpg"

    os.rename(old_file_name, new_file_name)
    
    #pull .jpeg into main folder rename .jpeg{i}
    
    i += 1
    
    # Wait for 1 second
    #time.sleep(1)

In [ ]:
import shutil
# define the name of the directory to be created
path = "static/data/raw/locs"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

In [ ]:
 
for root, dirs, files in os.walk('static/data/raw'):
    for file in files:
        if file.endswith('.jpg'):
            shutil.move(os.path.join(root, file),'static/data/raw/locs')

In [ ]:
import sys

!$sys.executable find .static/data/raw/locs -mindepth 1 ! -regex '^.static/data/raw/locs\(/.*\)?' -delete

In [ ]:
# # *** ORIGINAL PULL THAT WORKS ***
# # Define parameters for street view api
# params = [{
#     'size': '600x300', # max 640x640 pixels
#     'location': '41.9295582,-87.7159139',
#     'heading': '90',
#     'pitch': '-0.76',
#     'key': gkey
# }]

# # Create a results object
# results = google_streetview.api.results(params)

# # Download images to directory 'downloads'
# results.download_links('static/data/raw')

In [ ]:
## PRACTICING IMAGE PRE-PROCESSING

# # load and show an image with Pillow

# # load the image
# image = Image.open('static/data/test/brickornot.jpg')
# # summarize some details about the image
# print(image.format)
# print(image.mode)
# print(image.size)
# # show the image
# image.show()

In [ ]:
'''# load all images in a directory
loaded_images = list()
for filename in listdir('images'):
	# load image
	img_data = image.imread('images/' + filename)
	# store loaded image
	loaded_images.append(img_data)
	print('> loaded %s %s' % (filename, img_data.shape))'''


In [ ]:
%matplotlib inline
import PIL
from PIL import Image
from os import listdir
from matplotlib import image
from matplotlib import pyplot

# example of saving a grayscale version of a loaded image
from PIL import Image
# load the image
image = Image.open('static/data/raw/gsv_0.jpg')
# convert the image to grayscale
gs_image = image.convert(mode='L')
# report the size of the image
print(image.size)
# create a thumbnail and preserve aspect ratio
image.thumbnail((100,100))
# report the size of the thumbnail
print(image.size)
# save in jpeg format
gs_image.save('static/data/test_convert/gsv_0.jpg')

In [ ]:
# load the image again and show it
image2 = Image.open('static/data/test_convert/gsv_0.jpg')
# show the image
image2.show()

In [ ]:
'''# save as PNG format
image.save('opera_house.png', format='PNG')
# load the image again and inspect the format
image2 = Image.open('opera_house.png')
print(image2.format)'''